In [111]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [112]:
leetcode = pd.read_csv("leetcode_dataset - lc.csv")

In [113]:
leetcode = leetcode.drop(columns=["id", "title", "solution_link", "is_premium","similar_questions","related_topics", "url", "companies"])

In [114]:
print(leetcode.isna().sum())


description        0
difficulty         0
acceptance_rate    0
frequency          0
discuss_count      0
accepted           0
submissions        0
likes              0
dislikes           0
rating             0
asked_by_faang     0
dtype: int64


In [115]:
def parse_number(num_str):
    """Parse a string representing a number with scaling factor"""
    scaling_factors = {"K": 1000, "M": 1000000}
    last_char = num_str[-1]
    if last_char in scaling_factors:
        scaling_factor = scaling_factors[last_char]
        num = float(num_str[:-1])
        return num * scaling_factor
    else:
        return float(num_str)

# Example usage
num1 = parse_number("1.5M")   # Returns 1500000.0
num2 = parse_number("70.2K")  # Returns 70200.0


In [116]:
leetcode["accepted"] = leetcode["accepted"].apply(parse_number)
leetcode["submissions"] = leetcode["submissions"].apply(parse_number)

In [117]:
le = LabelEncoder()
leetcode["difficulty"] = le.fit_transform(leetcode["difficulty"])

In [118]:
X_train, X_test, y_train, y_test = train_test_split(leetcode["description"], leetcode["difficulty"], test_size=0.2, random_state=42)

In [119]:
vectorizer = CountVectorizer(stop_words="english")
X_train_desc = vectorizer.fit_transform(X_train)
X_test_desc = vectorizer.transform(X_test)

In [120]:
X_train = pd.DataFrame(X_train_desc.toarray())
X_test = pd.DataFrame(X_test_desc.toarray())


In [121]:
# Convert column names to string data type
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)


In [122]:
print(X_train.shape)
print(y_train.shape)

(1460, 6325)
(1460,)


In [123]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

/opt/anaconda3/envs/leetcode/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [124]:
y_pred = clf.predict(X_test)

In [125]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.31      0.36      0.33        89
           1       0.30      0.28      0.29        76
           2       0.58      0.56      0.56       200

    accuracy                           0.45       365
   macro avg       0.40      0.40      0.40       365
weighted avg       0.45      0.45      0.45       365

[[ 32  13  44]
 [ 17  21  38]
 [ 54  35 111]]


In [136]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix




# Remove irrelevant columns
leetcode = leetcode.drop(columns=["id", "title", "solution_link", "is_premium","similar_questions","related_topics", "url", "companies"], axis=1)
# Encode categorical variables
le = LabelEncoder()
leetcode["difficulty"] = le.fit_transform(leetcode["difficulty"])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(leetcode.drop(columns=["difficulty"]), leetcode["difficulty"], test_size=0.2, random_state=42)

# Define pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words="english")),
    ('clf', LogisticRegression())
])

# Define hyperparameters to tune
parameters = {
    'vect__max_features': [5000, 10000],
    'vect__ngram_range': [(1,1), (1,2)],
    'clf__C': [0.1, 1, 10]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)
grid_search.fit(X_train["description"], y_train)

# Print best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best F1 score:", grid_search.best_score_)

# Predict difficulty for testing set using best model
y_pred = grid_search.predict(X_test["description"])

# Evaluate performance using classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


/opt/anaconda3/envs/leetcode/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/leetcode/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

Best parameters: {'clf__C': 0.1, 'vect__max_features': 10000, 'vect__ngram_range': (1, 2)}
Best F1 score: 0.5082191780821917
              precision    recall  f1-score   support

           0       0.42      0.44      0.43        89
           1       0.33      0.22      0.27        76
           2       0.61      0.67      0.64       200

    accuracy                           0.52       365
   macro avg       0.45      0.44      0.44       365
weighted avg       0.50      0.52      0.51       365

[[ 39   9  41]
 [ 14  17  45]
 [ 40  26 134]]


NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [132]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Load dataset
df = pd.read_csv("leetcode_dataset - lc.csv")

# Remove irrelevant columns
df = df.drop(columns=["id", "title", "solution_link", "url", "similar_questions", "related_topics"])

# Encode categorical variables
le = LabelEncoder()
df["difficulty"] = le.fit_transform(df["difficulty"])

# Extract features from problem description
vectorizer = CountVectorizer(stop_words="english")
X_desc = vectorizer.fit_transform(df["description"])

# Combine description features with other features
X = pd.concat([df.drop(columns=["description"]), pd.DataFrame(X_desc.toarray())], axis=1)

# Train a logistic regression model
clf = LogisticRegression()
clf.fit(X, df["difficulty"])

# Take user input for problem description
problem_desc = input("Enter the problem description: ")

# Transform the input into a feature vector
input_desc = vectorizer.transform([problem_desc])
input_features = pd.DataFrame(input_desc.toarray())

# Predict difficulty for the input problem
difficulty = clf.predict(input_features)[0]
difficulty_str = le.inverse_transform([difficulty])[0]

print("The predicted difficulty of the problem is:", difficulty_str)


TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.